In [ ]:
#Script imports 
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import xlrd
import xlwt
from xlutils.copy import copy
import pandas as pd
from textblob import TextBlob
import re
import sys, io
import os, codecs
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)


In [ ]:
#!pip install xlutils

In [ ]:
#sentiment analysis
def processTweet2(tweet):
    # process the tweets

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet    


def get_sentiment(text):
    clean_text = processTweet2(text)
    analysis = TextBlob(clean_text)
    if analysis.sentiment[0]>0:
        senti = 'positive'
    elif analysis.sentiment[0]<0:
        senti = 'negative'
    else:
        senti = 'neutral'
    return senti    

In [ ]:
def saveWorkSpace(data):
    fields = json.loads(data)
    sentiment = get_sentiment(fields["text"])
    rb = xlrd.open_workbook('sentiment_analysis.xlsx')
    r_sheet = rb.sheet_by_index(0) 
    r = r_sheet.nrows
    wb = copy(rb) 
    sheet = wb.get_sheet(0) 
    sheet.write(r,0,fields['created_at'])
    sheet.write(r,1,fields['id'])
    sheet.write(r,2,fields["text"])
    sheet.write(r,3,sentiment)
    wb.save('sentiment_analysis.xlsx')
    

In [ ]:
#consumer key, consumer secret, access token, access secret.
ckey = 'wioFMRZqMchP22Wc8wbX32rdA'
csecret = 'DJE1aGOwbrqolwqwbciJ4zjSwalS0KDw2DwQ6xJPXTh3ZEfaSc'
atoken = '1022443048394006530-kEHquwcYkuErCS8oiasiaWmxd5BPxT'
asecret = 'recu23ACKlnnybvAKgIzavnOHa8IGSnauzVrkDO0Q2Dk1'

class listener(StreamListener):

    def on_data(self, data):
      print(data)
      saveWorkSpace(data)
      return(True)

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["M_Farmaajo"])

{"created_at":"Fri Sep 18 05:35:38 +0000 2020","id":1306829211894796288,"id_str":"1306829211894796288","text":"RT @CNN: Narendra Modi is seen by many as a \"national messiah\" who is working on a grander agenda to reshape the Indian nation along Hindu\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1128741283705475073,"id_str":"1128741283705475073","name":"V K","screen_name":"WittyWinay","location":null,"url":null,"description":null,"translator_type":"none","protected":false,"verified":false,"followers_count":339,"friends_count":218,"listed_count":0,"favourites_count":9386,"statuses_count":3074,"created_at":"Wed May 15 19:17:47 +0000 2019","utc_offset":null,"time_zone":null,"geo_enabled":false,"lang":null,"contributors_enabl

KeyboardInterrupt: ignored